In [ ]:
import pandas as pd
import os 
import requests
import time 
import pickle
from tqdm import tqdm
import numpy as np
from geopy.geocoders import Nominatim
import hashlib
import json
import sys
sys.path.append("/Users/eunkyungcha/GitHub/london_neighbourhood_recommendation/london_neighbourhood_recommendation/api")
from api_geocode import reverse_geocode
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")


# London Postcodes 

In [69]:
df_post = pd.read_csv("data/postcode_v0.csv")

df_post.columns = df_post.columns.str.strip().str.lower().str.replace(" ","_")
columns_to_keep = ["postcode", "county", "district", "ward", "population", "households","nearest_station", "police_force", "average_income", "property_type", "roads"]
df_post = df_post[columns_to_keep]

df_post.to_csv("data/postcode_v1_cleaned.csv")


/var/folders/1r/sjq4vmyj1tjf8k573fp7r6_40000gn/T/ipykernel_95585/2273394862.py:1: DtypeWarning: Columns (44) have mixed types. Specify dtype option on import or set low_memory=False.
  df_post = pd.read_csv("data/postcode_v0.csv")


In [70]:
columns_to_keep = ["postcode","district","ward","nearest_station"]
df_post = df_post[columns_to_keep]
df_post.to_csv("data/postcode_v2_cleaned.csv")

# Housing Price in London

In [44]:
# initial cleaning
# using the sale_estimate from 2024 analysis to reflect the realistic price 
df_house = pd.read_csv("data/house_v0.csv")

df_house.columns = df_house.columns.str.strip().str.lower().str.replace(" ","_")
columns_to_keep = ["postcode","outcode","latitude","longitude","bathrooms","bedrooms","floorareasqm","livingrooms","tenure","propertytype","saleestimate_currentprice"]
df_house = df_house[columns_to_keep]

df_house.to_csv("data/house_v1_cleaned.csv")



In [45]:
# property type category
bungalow_types = ["Bungalow Property", "Detached Bungalow", "End Terrace Bungalow",
    "Mid Terrace Bungalow", "Semi-Detached Bungalow", "Terraced Bungalow",
    "Terrace Property", "Semi-Detached Property", "Terraced"]

house_types = [
    "Detached House", "End Terrace House", "Semi-Detached House",
    "Mid Terrace House", "End Terrace Property", "Mid Terrace Property", 
    "Detached Property"
]

flat_types = ["Converted Flat", "Flat/Maisonette", "Purpose Built Flat"]

# convert 

def convert_property_type(ptype):
    if ptype in bungalow_types:
        return "Bungalow"
    elif ptype in house_types:
        return "House"
    elif ptype in flat_types:
        return "Flat"
    else:
        return "Other"
    
df_house["propertytype_converted"] = df_house["propertytype"].apply(convert_property_type)
columns_to_drop = ["propertytype"]
df_house = df_house.drop(columns = columns_to_drop)

df_house.to_csv("data/house_v2_propertyType_converted.csv", index=False)



In [46]:
# merge the postcode dataset to the house dataset - join on postcode 
# to get the area name in each postcode area 

df_house_merged = df_house.merge(df_post, on = "postcode", how = "left")

column_order = [
    "postcode", "outcode", "district", "ward", "livingrooms",
    "bedrooms", "bathrooms", "floorareasqm", "tenure", "propertytype_converted",
    "saleestimate_currentprice", "nearest_station","latitude","longitude"
]

df_house_merged = df_house_merged[column_order]

df_house_merged.to_csv("data/house_v3_mergedWithPost.csv")




In [47]:
# aggregate by outcode + ward 

group_cols = ["outcode","ward","livingrooms","bedrooms","bathrooms","tenure","propertytype_converted"]
agg_col = "saleestimate_currentprice"

df_house_grouped  = df_house_merged.dropna(subset = group_cols + [agg_col])

df_house_agg = (df_house_grouped.groupby(group_cols).agg(
    median_price=(agg_col, "median"),
    district = ("district","first"),
    nearest_station = ("nearest_station","first"))).reset_index() 

column_order = [
    "outcode","ward","district","livingrooms","bedrooms","bathrooms","tenure","propertytype_converted","median_price","nearest_station"
]

df_house_agg = df_house_agg[column_order]
df_house_agg = df_house_agg.dropna(subset=["outcode", "ward"])

df_house_agg.to_csv("data/house_v4_agg.csv")

# Crime Report

In [109]:
folder_path = "data/2024_crime_report"
all_crime_data = []

for month_folder in sorted(os.listdir(folder_path)):
    month_folder_path = os.path.join(folder_path, month_folder)
    if os.path.isdir(month_folder_path):
        monthly_data = []
        
        for filename in sorted(os.listdir(month_folder_path)):
            if filename.endswith(".csv"):
                file_path = os.path.join(month_folder_path, filename)
                print(f"Loading {filename}...")
                df_crime = pd.read_csv(file_path)
                print("Columns:", list(df_crime.columns))
                monthly_data.append(df_crime)
        
        if monthly_data:
            first_columns = monthly_data[0].columns
            for i, df_crime in enumerate(monthly_data):
                if not df_crime.columns.equals(first_columns):
                    print(f"Columns do not match in {month_folder}, file {i}: {df_crime.columns}")
            df_month_combined = pd.concat(monthly_data, ignore_index = True)
            all_crime_data.append(df_month_combined)
                    

if not all_crime_data:
    print("csv file not found.")
else:
    first_columns = all_crime_data[0].columns
    for i, df_crime in enumerate(all_crime_data):
        if not df_crime.columns.equals(first_columns):
            print(f"Columns do not match in file {i}: {df_crime.columns}")

df_crime = pd.concat(all_crime_data, ignore_index=True)

print("combined shape:", df_crime.shape)
print(df_crime.head())

df_crime.to_csv("data/crime_v1_combined.csv", index = False)




Loading 2024-01-city-of-london-street.csv...
Columns: ['Crime ID', 'Month', 'Reported by', 'Falls within', 'Longitude', 'Latitude', 'Location', 'LSOA code', 'LSOA name', 'Crime type', 'Last outcome category', 'Context']
Loading 2024-01-metropolitan-street.csv...
Columns: ['Crime ID', 'Month', 'Reported by', 'Falls within', 'Longitude', 'Latitude', 'Location', 'LSOA code', 'LSOA name', 'Crime type', 'Last outcome category', 'Context']
Loading 2024-02-city-of-london-street.csv...
Columns: ['Crime ID', 'Month', 'Reported by', 'Falls within', 'Longitude', 'Latitude', 'Location', 'LSOA code', 'LSOA name', 'Crime type', 'Last outcome category', 'Context']
Loading 2024-02-metropolitan-street.csv...
Columns: ['Crime ID', 'Month', 'Reported by', 'Falls within', 'Longitude', 'Latitude', 'Location', 'LSOA code', 'LSOA name', 'Crime type', 'Last outcome category', 'Context']
Loading 2024-03-city-of-london-street.csv...
Columns: ['Crime ID', 'Month', 'Reported by', 'Falls within', 'Longitude', 'Lat

In [144]:
df_crime.columns = df_crime.columns.str.lower().str.replace(" ", "_")

columns_to_keep = [
    "month", "reported_by", "longitude", "latitude", 
    "location", "crime_type"
]

df_crime = df_crime[columns_to_keep]

df_crime = df_crime.dropna(subset = ["longitude","latitude"])

df_crime = df_crime[~(df_crime["location"].str.strip() == "")]
df_crime = df_crime[~(df_crime["crime_type"].str.strip() == "")]

df_crime["month"] = pd.to_datetime(df_crime["month"])

print(df_crime.info())
print(df_crime.head())

df_crime.to_csv("data/crime_v2_cleaned.csv", index = False)



<class 'pandas.core.frame.DataFrame'>
Index: 1147685 entries, 0 to 1149320
Data columns (total 6 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   month        1147685 non-null  datetime64[ns]
 1   reported_by  1147685 non-null  object        
 2   longitude    1147685 non-null  float64       
 3   latitude     1147685 non-null  float64       
 4   location     1147685 non-null  object        
 5   crime_type   1147685 non-null  object        
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 61.3+ MB
None
       month            reported_by  longitude   latitude  \
0 2024-01-01  City of London Police  -0.106220  51.518275   
1 2024-01-01  City of London Police  -0.107682  51.517786   
2 2024-01-01  City of London Police  -0.111596  51.518281   
3 2024-01-01  City of London Police  -0.111596  51.518281   
4 2024-01-01  City of London Police  -0.112096  51.515942   

                   location             c

### longitude & latitude to postcodes


In [128]:
# google geocoding API takes a very long time to process, so we will use postcodes.io API instead 

def fetch_postcodes_postcodesio_batch(coords_batch):
    url = "https://api.postcodes.io/postcodes"
    payload = {"geolocations": [{"longitude": lng, "latitude": lat} for lat, lng in coords_batch]}
    try:
        response = requests.post(url, json=payload, timeout=10)
        data = response.json()
        results = data.get("result", [])
        postcodes = []
        for item in results:
            if item and item.get("result"):
                postcode = item["result"][0].get("postcode")
                postcodes.append(postcode)
            else:
                postcodes.append(None)
        return postcodes
    except Exception as e:
        print(f"Error during batch request: {e}")
        return [None] * len(coords_batch)

def add_postcode_using_postcodesio(df_crime, cache_pkl="coord_to_postcode.pkl"):
    lng_col = "longitude" if "longitude" in df_crime.columns else "longtitude"
    df_crime["lat_rounded"] = df_crime["latitude"].round(5)
    df_crime["lng_rounded"] = df_crime[lng_col].round(5)

    # unique coordinate set
    unique_coords = df_crime[["lat_rounded", "lng_rounded"]].drop_duplicates()
    coord_list = list(unique_coords.itertuples(index=False, name=None))

    print(f"Total unique coordinates to lookup: {len(coord_list)}")

    coord_to_postcode = {}
    batch_size = 100

    for i in tqdm(range(0, len(coord_list), batch_size)):
        batch = coord_list[i:i+batch_size]
        postcodes = fetch_postcodes_postcodesio_batch(batch)
        for coord, postcode in zip(batch, postcodes):
            coord_to_postcode[coord] = postcode
        time.sleep(0.05)  # setting the delay to 50ms to avoid hitting the rate limit

    # Saving the lookup dictionary for future reuse just in case
    with open(cache_pkl, "wb") as f:
        pickle.dump(coord_to_postcode, f)

    # Map using vectorised .map for speed
    df_crime["coord_key"] = list(zip(df_crime["lat_rounded"], df_crime["lng_rounded"]))
    df_crime["postcode"] = df_crime["coord_key"].map(coord_to_postcode)

    # Clean up
    df_crime.drop(columns=["lat_rounded", "lng_rounded", "coord_key"], inplace=True)

    return df_crime

if __name__ == "__main__":
    df_crime = pd.read_csv("data/crime_v2_cleaned.csv")

    # add postcodes to the dataset
    df_crime = add_postcode_using_postcodesio(df_crime, cache_pkl="data/coord_to_postcode.pkl")

    df_crime.to_csv("data/crime_v3_with_postcode.csv", index=False)
    print("Saved dataset with postcodes to data/crime_v3_with_postcode.csv")

Total unique coordinates to lookup: 60948


 45%|████▌     | 277/610 [07:28<22:36,  4.07s/it]

Error during batch request: HTTPSConnectionPool(host='api.postcodes.io', port=443): Read timed out. (read timeout=10)


100%|██████████| 610/610 [16:15<00:00,  1.60s/it]


Saved dataset with postcodes to data/crime_v3_with_postcode.csv


In [60]:
# merge the postcode dataset to the crime dataset 
# to get the area name in each postcode  
df_crime = pd.read_csv("data/crime_v3_with_postcode.csv")

df_crime_merged = df_crime.merge(df_post, on = ["postcode"], how = "left")

df_crime_merged = df_crime_merged.drop(columns = ["reported_by","longitude","latitude","location","nearest_station"])

# add outcode
df_crime_merged["outcode"] = df_crime_merged["postcode"].str.extract(r"^([A-Z]{1,2}\d{1,2}[A-Z]?)")

column_order = [
    "month","postcode","outcode","ward","district","crime_type"
]
df_crime_merged = df_crime_merged[column_order]

# drop all rows with non-London postcode
df_crime_merged = df_crime_merged.dropna(subset=["outcode","ward"])

df_crime_merged.to_csv("data/crime_v4_mergedWithPost.csv")



# Population 
for crime rate level categorisation & crime type count per outcode

In [78]:
# population dataset processing
df_pop = pd.read_csv("data/postcode_v0.csv")
df_pop.columns = df_pop.columns.str.strip().str.lower().str.replace(" ","_")

columns_to_keep = ["postcode","ward","population"]
df_pop = df_pop[columns_to_keep]

# add outcode 
df_pop["outcode"] = df_pop["postcode"].str.extract(r"^([A-Z]{1,2}\d{1,2}[A-Z]?)")
# remove postcode
df_pop = df_pop.drop(columns = ["postcode"])

column_order = ["outcode","ward","population"]
df_pop = df_pop[column_order]

df_pop.to_csv("data/population_v1_extractedFromPost.csv")




/var/folders/1r/sjq4vmyj1tjf8k573fp7r6_40000gn/T/ipykernel_95585/4053501997.py:2: DtypeWarning: Columns (44) have mixed types. Specify dtype option on import or set low_memory=False.
  df_pop = pd.read_csv("data/postcode_v0.csv")


In [79]:
df_pop_grouped = df_pop.groupby(["outcode","ward"], as_index = False)["population"].sum()
df_pop_grouped.rename(columns={"population":"total_population"}, inplace = True)

df_pop_grouped.to_csv("data/population_v2_agg.csv")

In [80]:
# join the crime report dataset using outcode
# crime rate per outcode & ward
df_crime = pd.read_csv("data/crime_v4_mergedWithPost.csv")
df_pop = pd.read_csv("data/population_v2_agg.csv")

# crime count per outcode & ward
df_crime_counts = df_crime.groupby(["outcode","ward"]).size().reset_index(name="crime_counts")

# merge with population data on outcode, then calculate the crime rate per 1000 ppl 
df_crime_level = df_crime_counts.merge(df_pop, on=["outcode","ward"], how="left")
df_crime_level["crime_rate_per_1000"] = (df_crime_level["crime_counts"]/df_crime_level["total_population"]) * 1000 

# categorise crime rate level 
def categorise_crime(rate):
    if pd.isna(rate):
        return "No info"
    elif rate < 140:
        return "Low crime"
    elif rate <= 225:
        return "Medium crime"
    else:
        return "High crime"
    

df_crime_level["crime_level"] = df_crime_level["crime_rate_per_1000"].apply(categorise_crime)


# top 3 crimes per outcode
df_top_crimes = (
    df_crime.groupby(["outcode","ward", "crime_type"])
    .size()
    .reset_index(name="count")
)

df_top3_crimes = (
    df_top_crimes.sort_values(["outcode","ward","count"], ascending=[True, True, False])
    .groupby(["outcode","ward"])
    .head(3)
)

df_top3_crimes["rank"] = df_top3_crimes.groupby(["outcode","ward"]).cumcount()+1
df_crime_rank = df_top3_crimes.pivot(index=["outcode","ward"],columns="rank", values="crime_type").reset_index()
df_crime_rank.columns = ["outcode","ward","crime_1","crime_2","crime_3"]

df_crime_combined = df_crime_merged.merge(df_crime_level, on=["outcode","ward"], how="left")
df_crime_combined = df_crime_combined.merge(df_crime_rank, on=["outcode","ward"], how="left")


df_crime_combined.to_csv("data/crime_v5_combined_stats.csv", index=False)
    




In [81]:
df_crime_combined = pd.read_csv("data/crime_v5_combined_stats.csv")

columns_to_keep = ["outcode","district","ward","crime_level","crime_1","crime_2","crime_3"]
df_crime_combined = df_crime_combined[columns_to_keep]

df_crime_combined = df_crime_combined.drop_duplicates(subset=["outcode","ward"])
df_crime_combined = df_crime_combined.dropna(subset=["outcode", "ward"])

df_crime_combined.to_csv("data/crime_v6_combined_cleaned.csv")

# Public School Quality

In [13]:
df_school = pd.read_csv(
    "data/school_v0.csv",
    encoding="ISO-8859-1"
)

df_school.columns = df_school.columns.str.lower().str.replace(" ", "_")

columns_to_keep = ["school_name","ofsted_phase","type_of_education","ungraded_inspection_overall_outcome","sixth_form","designated_religious_character","ofsted_region","local_authority","postcode","total_number_of_pupils","statutory_lowest_age","statutory_highest_age"]
df_school = df_school[columns_to_keep]

print(df_school["ofsted_region"].unique()) # unique values of each column
df_school = df_school[df_school["ofsted_region"] == "London"] # only include London

df_school = df_school.fillna("No info")

allowed_phrases = ["School remains Good", "School remains Outstanding", "No information"]
df_school = df_school[df_school["ungraded_inspection_overall_outcome"].isin(allowed_phrases)].copy()

# convert
df_school["ungraded_inspection_overall_outcome"] = df_school["ungraded_inspection_overall_outcome"].replace({
    "School remains Good": "Good",
    "School remains Outstanding": "Outstanding",
    "No information": "No information"
})

df_school.to_csv("data/school_v1_cleaned.csv")



['London' 'South East' 'North East, Yorkshire and the Humber'
 'West Midlands' 'North West' 'South West' 'East of England'
 'East Midlands']


/var/folders/1r/sjq4vmyj1tjf8k573fp7r6_40000gn/T/ipykernel_95585/3392353320.py:1: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df_school = pd.read_csv(


In [14]:
# merge
df_post = pd.read_csv("data/postcode_v2_cleaned.csv")
df_school_merged = df_school.merge(df_post, on = "postcode", how = "left")

# add outcode column and remove postcode column
df_school_merged["outcode"] = df_school_merged["postcode"].str.extract(r"^([A-Z]{1,2}\d{1,2}[A-Z]?)")
df_school_merged = df_school_merged.drop(columns=["postcode","nearest_station","ofsted_region","local_authority"])

# reorder the columns
column_order = ["outcode","ward","district","school_name","ofsted_phase","type_of_education","ungraded_inspection_overall_outcome","sixth_form","designated_religious_character","total_number_of_pupils","statutory_lowest_age","statutory_highest_age"]
df_school_merged = df_school_merged[column_order]


df_school_merged.to_csv("data/school_v2_mergedWithPost.csv")


In [16]:
print(df_school_merged.columns)

Index(['outcode', 'ward', 'district', 'school_name', 'ofsted_phase',
       'type_of_education', 'ungraded_inspection_overall_outcome',
       'sixth_form', 'designated_religious_character',
       'total_number_of_pupils', 'statutory_lowest_age',
       'statutory_highest_age'],
      dtype='object')


In [21]:
# group school data by good or outstanding schools - per outcode & ward

good_schools = df_school_merged[df_school_merged["ungraded_inspection_overall_outcome"] == "Good"]
outstanding_schools = df_school_merged[df_school_merged["ungraded_inspection_overall_outcome"] == "Outstanding"]

summary_good = (
    good_schools.groupby(["outcode","ward"]).agg(
    num_good = ("school_name", "count"),
    schools_good = ("school_name", lambda names: "; ".join(names)))
.reset_index()
)

summary_outstanding = (
    outstanding_schools.groupby(["outcode","ward"]).agg(
        num_outstanding = ("school_name","count"),
        schools_outstanding = ("school_name", lambda names: "; ".join(names)))
.reset_index()
)
 # merging 
school_summary = pd.merge(summary_good, summary_outstanding, on=["outcode","ward"], how="outer")

# filling empty cells 
school_summary["num_good"] = school_summary["num_good"].fillna(0).astype(int)
school_summary["num_outstanding"] = school_summary["num_outstanding"].fillna(0).astype(int)
school_summary["schools_good"] = school_summary["schools_good"].fillna("No info")
school_summary["schools_outstanding"] = school_summary["schools_outstanding"].fillna("No info")

school_summary = school_summary.dropna(subset=["outcode", "ward"])

school_summary.to_csv("data/school_v3_agg.csv")

# Combine all dataset on outcode 

In [82]:
df_crime = pd.read_csv("data/crime_v6_combined_cleaned.csv")
df_school = pd.read_csv("data/school_v3_agg.csv")
df_house = pd.read_csv("data/house_v4_agg.csv")

df_final = df_house.merge(df_crime, on = ["outcode","ward"], how = "left")
df_final = df_final.merge(df_school, on = ["outcode", "ward"], how = "left")

# avoid generating multiple district columns
if "district_x" in df_final.columns and "district_y" in df_final.columns:
    df_final = df_final.rename(columns = {"district_x":"district"})
    df_final = df_final.drop(columns = ["district_y"], errors = "ignore")
    
# drop index columns 
unnamed_cols = [col for col in df_final.columns if col.startswith("Unnamed")]
df_final = df_final.drop(columns = unnamed_cols)

# fill empty values 
df_final["num_good"] = df_final["num_good"].fillna(0).astype(int)
df_final["num_outstanding"] = df_final["num_outstanding"].fillna(0).astype(int)
df_final["schools_good"] = df_final["schools_good"].fillna("No info")
df_final["schools_outstanding"] = df_final["schools_outstanding"].fillna("No info")



df_final.to_csv("final_data/final_v1.csv")

In [ ]:
# replace the coordinates according to the area name and outcode

df_lat_lng = pd.read_csv("final_data/final_v1.csv")

df_lat_lng["query"] = df_lat_lng["ward"] + ", " + df_lat_lng["outcode"] + ", London, UK"
unique_queries = df_lat_lng["query"].dropna().unique()

def geocode_address(address, api_key):
    url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {
        "address": address,
        "key": api_key
    }
    response = requests.get(url, params=params)
    data = response.json()

    if data["status"] == "OK":
        location = data["results"][0]["geometry"]["location"]
        return location["lat"], location["lng"]
    else:
        print(f"Failed for {address}: {data.get('status')}")
        return None, None

# get latitude and longitude for each unique query
results = []
for i, query in enumerate(unique_queries):
    lat, lng = geocode_address(query, API_KEY)
    results.append({
        "query": query,
        "latitude": lat,
        "longitude": lng
    })
    time.sleep(0.1)
    if i % 25 == 0:
        print(f"{i} done...")

# convert to dataframe
df_coords = pd.DataFrame(results)

# merge back with original dataset 
df_combined = df_lat_lng.merge(df_coords, on="query", how="left")

# replace the old data with the new coordinates
df_combined = df_combined.drop(columns=["latitude_x", "longitude_x"], errors="ignore")
df_combined = df_combined.rename(columns={"latitude_y": "latitude", "longitude_y": "longitude"})


df_combined.to_csv("final_data/final_v2_latlng.csv", index=False)


0 done...
25 done...
50 done...
75 done...
100 done...
125 done...
150 done...
175 done...
200 done...
225 done...
250 done...
275 done...
300 done...
325 done...
350 done...
375 done...
400 done...
425 done...
450 done...
475 done...
500 done...
525 done...
550 done...
575 done...
600 done...
625 done...
650 done...
675 done...
700 done...
725 done...
750 done...
775 done...
800 done...
825 done...
850 done...
875 done...


In [84]:
df_combined = pd.read_csv("final_data/final_v2_latlng.csv")
df_combined = df_combined.drop(columns = ["Unnamed: 0","query"])

# numeric and string columns
numeric_columns = [
  "livingrooms",
  "bedrooms",
  "bathrooms",
  "median_price",
  "num_good",
  "num_outstanding",
  "latitude",
  "longitude"
]

string_columns = [
    "outcode",
    "ward",
    "district",
    "tenure",
    "propertytype_converted",
    "nearest_station",
    "crime_level",
    "crime_1",
    "crime_2",
    "crime_3",
    "schools_good",
    "schools_outstanding"
]


for col in numeric_columns:
    df_combined[col] = pd.to_numeric(df_combined[col], errors="coerce")  

for col in string_columns:
    df_combined[col] = df_combined[col].replace(["", " ", None, np.nan], "No info")
    

df_combined.to_csv("final_data/final_v3_datatype.csv")



In [85]:
df_combined.isnull().sum()

outcode                   0
ward                      0
district                  0
livingrooms               0
bedrooms                  0
bathrooms                 0
tenure                    0
propertytype_converted    0
median_price              0
nearest_station           0
crime_level               0
crime_1                   0
crime_2                   0
crime_3                   0
num_good                  0
schools_good              0
num_outstanding           0
schools_outstanding       0
latitude                  0
longitude                 0
dtype: int64